In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
import seaborn as sns

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=2.2)

## Problema (Blocos)

<font size=3px>O sistema representado na figura está em equilíbrio e θ = $60^\circ$.<br>
Considere agora que o fio 2 é cortado. Após algum tempo, cada um dos blocos oscilará como
um pêndulo. Se o comprimento do fio 1 é 10 cm, qual será a velocidade máxima atingida pelo
bloco de 100 kg durante seu movimento de oscilação? Despreze forças de atrito.<br>
</font>
<img src="blocosTensoes.jpg" alt="Drawing" style="width: 250px;"/>

## Problema (Pêndulo)

<font size=3px>Um pêndulo feito de uma esfera de massa $m$ presa a um fio de massa 
    desprezível e comprimento $L$, é solto do ponto $A$, conforme mostra a figura. <br>
    a) Qual a velocidade da esfera no ponto B?<br>
    b) Qual a tração da corda quando ela está neste ponto?
</font>
<img src="pendulo.png" alt="Drawing" style="width: 250px;"/>

## Problema (Mola com atrito)

<font size=3px>Uma mola de constante elástica $k = 20$ N/m está presa a um bloco de massa $m = 5$ kg. O bloco é
    colocado na posição $x_0 = -1$ m e solto a partir do repouso. A mola encontra-se relaxada quando a posição do bloco
    é $x=0$. Se a superfície possui um coeficiente de atrito cinético $\mu_k = 0.01$, determine:<br>
    a) a posição máxima do bloco,<br>
    b) a energia cinética máxima do bloco.
</font>